# Mini projet Qualité de Données : Détections des doublons
## ***Christophe COMPAIN / Sander COHEN***

### Objectif et Données Disponibles
L'objectif du projet est d'identifier les logiciels vendus sur les deux plateformes.
Pour ce faire, nous disposons des données pour chacune des plateformes isolément, respectivement dans les fichiers ***Company1.csv*** et ***Company2.csv***.

### Import packages, Variables Globales et import csv

In [1]:
import pandas as pd
import nltk
import time
import numpy as np
import math
import re
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ccompain/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ccompain/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
#path = "D:\\OneDrive - Université Paris-Dauphine\\Bureau\\Cours Master\\12-Qualité de Données\\\Projet\\mini-projet\\"
path = "/Users/ccompain/Documents/code/Dauphine/MasterIASD_prepa_data/mini-projet/github/"
#file1= "Data\\Company1.csv" #"SampleData\\Sample_Company1.csv"
file1 = "Data/Company1.csv"
#file2= "Data\\Company2.csv" #"SampleData\\Sample_Company2.csv"
file2 = "Data/Company2.csv"
#real= "Data\\Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"
real= "Data/Ground_truth_mappings.csv" #"SampleData\\Sample_Groud_truth_mappings.csv"

In [3]:
company1 = pd.read_csv(path+file1, encoding = "ISO-8859-1")
company2 = pd.read_csv(path+file2, encoding = "ISO-8859-1")
ground_truth_matches = pd.read_csv(path+real, encoding = "ISO-8859-1").drop_duplicates()

### Exploration des données

#### Aspects généraux

In [4]:
company1.head(5)

,id,title,description,manufacturer,price
0,b000jz4hqo,clickart 950 000 - premier image pack (dvd-rom),NaN,broderbund,0.00
1,b0006zf55o,ca international - arcserve lap/desktop oem 30pk,oem arcserve backup v11.1 win 30u for laptops ...,computer associates,0.00
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.00
3,b000g80lqo,peachtree by sage premium accounting for nonpr...,peachtree premium accounting for nonprofits 20...,sage software,599.99
4,b0006se5bq,singing coach unlimited,singing coach unlimited - electronic learning ...,carry-a-tune technologies,99.99


In [5]:
company1.count()

id              1363
title           1363
description     1248
manufacturer    1363
price           1363
dtype: int64

In [6]:
company2.head(5)

,id,name,description,manufacturer,price
0,11125907881740407428,learning quickbooks 2007,learning quickbooks 2007,intuit,38.99
1,11538923464407758599,superstart! fun with reading & writing!,fun with reading & writing! is designed to hel...,NaN,8.49
2,11343515411965421256,qb pos 6.0 basic software,qb pos 6.0 basic retail mngmt software. for re...,intuit,637.99
3,12049235575237146821,math missions: the amazing arcade adventure (g...,save spectacle city by disrupting randall unde...,NaN,12.95
4,12244614697089679523,production prem cs3 mac upgrad,adobe cs3 production premium mac upgrade from ...,adobe software,805.99


In [7]:
company2.count()

id              3226
name            3226
description     3035
manufacturer     232
price           3226
dtype: int64

In [8]:
ground_truth_matches.head(5)

,idCompany1,idCompany2
0,b000jz4hqo,18441480711193821750
1,b00004tkvy,18441110047404795849
2,b000g80lqo,18441188461196475272
3,b0006se5bq,18428750969726461849
4,b00021xhzw,18430621475529168165


In [9]:
ground_truth_matches.count()

idCompany1    1300
idCompany2    1300
dtype: int64

#### Top manufacturers

In [10]:
company1.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
adobe                   87
encore software         76
topics entertainment    73
encore                  62
microsoft               58
aspyr media             27
apple                   26
fogware publishing      19
intuit                  18
Name: id, dtype: int64

In [11]:
company2.groupby('manufacturer')['id'].count().sort_values(ascending=False).head(9)

manufacturer
punch software         30
abacus software        27
freeverse software     24
individual software    24
apple software         15
intuit                  8
webroot software        7
onone software          7
ce software             6
Name: id, dtype: int64

#### Observation d'un premier duplicat

In [12]:
company1[company1.id == ground_truth_matches.idCompany1[1]]

,id,title,description,manufacturer,price
2,b00004tkvy,noah's ark activity center (jewel case ages 3-8),NaN,victory multimedia,0.0


In [13]:
company2[company2.id == ground_truth_matches.idCompany2[1]]

,id,name,description,manufacturer,price
1881,18441110047404795849,the beginners bible: noah's ark activity cente...,NaN,NaN,9.95


### Récupération d'une base de stop_words pour nettoyage des descriptions

In [14]:
stop_words = set(nltk.corpus.stopwords.words('english'))
stop_words.update(["r","v","software","entertainment","inc","usa"])

### Fonction de tokenization et nettoyage de chaine de caractères
 - suppression des caractères non alpha
 - conversion majuscule vers minuscule
 - remplacement d'abbréviations communes
 - lemmatisation

In [15]:
def prep(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    texte = re.sub("[^a-zA-Z0-9_]", " ",str(texte)).lower()
    #remplacement de mots
    texte = texte.replace("professional", "pro").replace(" win "," windows ").replace(" upg "," upgrade ").replace(" dlx "," deluxe ")
    #tokenization par mot
    tokens = nltk.word_tokenize(texte)
    #suppression des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_words]
#    # Stemming
#    texte = [nltk.stem.SnowballStemmer('english').stem(w) for w in filtered_tokens]
    # Lemmatization
    texte = [nltk.stem.WordNetLemmatizer().lemmatize(w) for w in filtered_tokens]
    #remise sous forme d'une string
    return " ".join(texte)

### Fonction de normalisation des prix
 - suppression caractères non-alpha
 - conversion en minuscule

In [16]:
def retreatprice(texte):
    #suppression des caracteres non alphanumériques + tout en minuscule
    return float(re.sub("[^0-9.]", " ",str(texte)))

### Pre-Processing des 2 dataframes
 - Renommage des colonnes
 - Imputation des "na"
 - Traitement des prix
 - Construction d'une colonne "full data" contenant le titre et le manufacturer sur laquelle la fonction "prep" est appliquée
 - Ajout d'une colonne de version de l'OS

In [17]:
company1['Company']="company1"
company1=company1.rename(columns={"title": "name"})
company1['name'] = company1['name'].fillna(' ')
company1['manufacturer'] = company1['manufacturer'].fillna(' ')
company1['description'] = company1['description'].fillna(' ')
company1['price'] = company1['price'].fillna(' ')
company1['price_retreat'] = company1['price'].apply(retreatprice)
company1['full data']=company1['manufacturer'].apply(prep) + ' ' + company1['name'].apply(prep) # + ' ' + company1['description'].apply(prep)

company2['Company']="company2"
company2['name'] = company2['name'].fillna(' ')
company2['manufacturer'] = company2['manufacturer'].fillna(' ')
company2['description'] = company2['description'].fillna(' ')
company2['price'] = company2['price'].fillna(' ')
company2['price_retreat'] = company2['price'].apply(retreatprice)
company2['full data']=company2['manufacturer'].apply(prep) + ' ' + company2['name'].apply(prep) # + ' ' + company2['description'].apply(prep)

In [18]:
#Ajout d'une colonne de version
company1['Win']=np.where(company1['name'].str.contains('window') | company1['description'].str.contains('window'),1,0)
company1['Mac']=np.where(company1['name'].str.contains('mac') | company1['description'].str.contains('mac'),1,0)
company1['Linux']=np.where(company1['name'].str.contains('linux') | company1['description'].str.contains('linux'),1,0)
company1['version']=np.select(
    [(company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Win']==1),
     (company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Mac']==1),
     (company1['Win']+company1['Mac']+company1['Linux']==1) & (company1['Linux']==1),
     (company1['Win']+company1['Mac']+company1['Linux']!=1)
    ],
    ['Win','Mac','Linux','None'])

company2['Win']=np.where(company2['name'].str.contains('window') | company2['description'].str.contains('window'),1,0)
company2['Mac']=np.where(company2['name'].str.contains('mac') | company2['description'].str.contains('mac'),1,0)
company2['Linux']=np.where(company2['name'].str.contains('linux') | company2['description'].str.contains('linux'),1,0)
company2['version']=np.select(
    [(company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Win']==1),
     (company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Mac']==1),
     (company2['Win']+company2['Mac']+company2['Linux']==1) & (company2['Linux']==1),
     (company2['Win']+company2['Mac']+company2['Linux']!=1)
    ],
    ['Win','Mac','Linux','None'])

### Création d'un Dataframe "corpus" contenant les données de company1 et company2

In [19]:
corpus = pd.concat([company1, company2],sort=False,ignore_index=True)
#corpus.reset_index(drop=True)

In [20]:
len(corpus)

4589

In [21]:
corpus.tail()

,id,name,description,manufacturer,price,Company,price_retreat,full data,Win,Mac,Linux,version
4584,14872602878188858026,jumpstart(r) advanced 1st grade,prepare your child for the 1st grade and beyon...,,19.99,company2,19.99,jumpstart advanced 1st grade,0,0,0,None
4585,14916162814320983138,ibm(r) viavoice(r) advanced edition 10,ibm viavoice advanced edition release 10 is a ...,,78.95,company2,78.95,ibm viavoice advanced edition 10,1,0,0,Win
4586,14974113209571399013,xbox 360: gears of war,as marcus fenix you fight a war against the im...,,59.99,company2,59.99,xbox 360 gear war,0,0,0,None
4587,14986935400648190776,documents to go premium 7.0,this pda software enables you to use your docu...,,49.99,company2,49.99,document go premium 7 0,0,0,0,None
4588,14996991014087320062,microsoft(r) picture it! digital image pro 9.0,picture it! digital image pro puts you in cont...,,99.87,company2,99.87,microsoft picture digital image pro 9 0,0,0,0,None


In [22]:
###SUPPRIME : ne donne pas les resultats escomptés
###recherche des mots unique pour les supprimer
allwords = corpus['full data'].str.split(expand=True).stack().value_counts()
stop_unique = set(allwords[allwords==1].index)
def prep2(texte):
    tokens = nltk.word_tokenize(texte)
    #supreesion des stopwords
    filtered_tokens = [w for w in tokens if not w in stop_unique]
    #remise sous forme d'une string
    return " ".join(filtered_tokens)
#company1['full data']=company1['full data'].apply(prep2)
#company2['full data']=company2['full data'].apply(prep2)

### Fonction d'identification des doublons
 Après quelques essais pas forcément concluants utilisants la distance de Jacquard, nous avons décidés d'expérimenter TF-IDF
 - TF : Term-Frequency => Renvoie la fréquence de chaque mot dans le corpus
 - IDF : Inverse-Data-Frequency => Calcule du poids des mots
 - le TF-IDF score sera le produit des 2 précédentes valeurs

Nous avons également utilisé le "prix" comme critère de proximité :
 - `(price1* price2 == 0 or max(price1, price2)/min(price1, price2)<2)`

In [23]:
#fonction tfidf
def filtre_tfidf(corpus, ngram, max_df,sim_lim, stop):
    global number_of_matches, matches, matches_df
    start = time.process_time()
    new_number_of_matches = 0
    new_matches=[]

    vectorizer = TfidfVectorizer(ngram_range=ngram, max_df=max_df,sublinear_tf=True,stop_words=[stop]) 
    vectors = vectorizer.fit_transform(corpus['full data'])
    feature_names = vectorizer.get_feature_names()
    dense = vectors.todense()

    for i in range(len(company1_light)):
        #try :  
        price1 = float(company1_light.iloc[i,6]) 
        #except : 
        #    price1 = 0
        #tokens1name = nltk.word_tokenize(company1_light.iloc[i,7])
        for j in range(len(company2_light)):
            #try :  
            price2 = float(company2_light.iloc[j,6]) 
            #except : 
            #    price2 = 0
            #tokens2name = nltk.word_tokenize(company2_light.iloc[j,7])
            #ng2_tokensname = set(nltk.ngrams(tokens2name, n=1))
            #jd_ng1_ng2_name = nltk.jaccard_distance(ng1_tokensname, ng2_tokensname)
            if (price1* price2 == 0 or max(price1, price2)/min(price1, price2)<2) and (company1_light.iloc[i,11]==company2_light.iloc[j,11] or company1_light.iloc[i,11]=='None' or company2_light.iloc[j,11]=='None'):
                try :
                    similarity = np.dot(dense[i],np.transpose(dense[len(company1_light)+j])).item(0)/math.sqrt(np.dot(dense[i],np.transpose(dense[i])).item(0) * np.dot(dense[len(company1_light)+j],np.transpose(dense[len(company1_light)+j])).item(0))
                except : 
                    similarity = 0
                if  similarity > sim_lim: #or jd_ng1_ng2_name<0.2 :# or name_score<=1) :
                    new_number_of_matches = new_number_of_matches +1
                    new_matches.append((company1_light.iloc[i,0],company2_light.iloc[j,0]))

    if new_number_of_matches>0:
        print("New matches: {}".format(new_number_of_matches))
        number_of_matches= number_of_matches + new_number_of_matches
        print("Total matches: {}".format(number_of_matches))
        if matches== []:
            matches = new_matches 
            matches_df = pd.DataFrame(matches)
            matches_df.columns= ['idCompany1','idCompany2']
        else:
            new_matches_df = pd.DataFrame(new_matches)
            new_matches_df.columns= ['idCompany1','idCompany2']
            matches_df = pd.concat([matches_df, new_matches_df],sort=False,ignore_index=True).drop_duplicates()

        diff_df = pd.merge(ground_truth_matches, matches_df, how='outer', indicator='Exist')
        true_positives = diff_df[diff_df.Exist=='both']
        false_positives = diff_df[diff_df.Exist=='right_only']
        false_negatives = diff_df[diff_df.Exist=='left_only']
        print("Number of true positives: {}".format(len(true_positives)))
        print("Number of false positives: {}".format(len(false_positives)))
        print("Number of false negatives: {}".format(len(false_negatives)))
        precision = len(true_positives)/(len(true_positives)+ len(false_positives))
        print("Precision: {}".format(precision))
        recall = len(true_positives)/(len(true_positives)+ len(false_negatives))
        print("Recall: {}".format(recall))
        try :
            f_measure = 2*(precision*recall)/(precision+recall)
            print("F measure: {}".format(f_measure))
        except:
            print("F measure not calculable")
    else:
        print("No new match")
    end = time.process_time()
    print("Processing time: {}".format(end - start))

In [24]:
company1_light=company1
company2_light=company2

In [25]:
#boucle sur les big manufacturer
big_manufacturer_list=["microsoft","apple","encore","adobe","punch","topic","aspyr","sony","webroot"]
number_of_matches = 0
matches=[]
matches_df=[]
for filtre in big_manufacturer_list :
    for max_ngram in [2,1]:
        try :
            company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
            company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
        except : 
            company1_light=company1
            company2_light=company2        
        company1_light = company1_light[company1_light['full data'].str.contains(filtre)].reset_index(drop=True)
        company2_light = company2_light[company2_light['full data'].str.contains(filtre)].reset_index(drop=True)
        corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
        print("{} with ngram=(1,{})".format(filtre,max_ngram))
        filtre_tfidf(corpus, (1,max_ngram), 0.1 ,0.6, filtre)
        print(" ")

microsoft with ngram=(1,2)
New matches: 36
Total matches: 36
Number of true positives: 22
Number of false positives: 14
Number of false negatives: 1278
Precision: 0.6111111111111112
Recall: 0.016923076923076923
F measure: 0.03293413173652695
Processing time: 1.2501489999999995
 
microsoft with ngram=(1,1)
New matches: 13
Total matches: 49
Number of true positives: 29
Number of false positives: 20
Number of false negatives: 1271
Precision: 0.5918367346938775
Recall: 0.022307692307692306
F measure: 0.04299481097108969
Processing time: 0.5417419999999993
 
apple with ngram=(1,2)
New matches: 19
Total matches: 68
Number of true positives: 43
Number of false positives: 25
Number of false negatives: 1257
Precision: 0.6323529411764706
Recall: 0.03307692307692308
F measure: 0.06286549707602339
Processing time: 0.16066499999999984
 
apple with ngram=(1,1)
New matches: 12
Total matches: 80
Number of true positives: 53
Number of false positives: 27
Number of false negatives: 1247
Precision: 0.662

In [26]:
##boucle sur les autres noms
for max_ngram in [3,2,1]:
    company1_light=company1[~company1.id.isin(matches_df.idCompany1)]
    company2_light=company2[~company2.id.isin(matches_df.idCompany2)]    
    corpus = pd.concat([company1_light, company2_light],sort=False,ignore_index=True)
    print("Other entries with ngram=(1,{})".format(max_ngram))
    filtre_tfidf(corpus, (1,max_ngram), 0.01 ,0.5, "")
    print(" ")




Other entries with ngram=(1,3)
New matches: 578
Total matches: 1008
Number of true positives: 644
Number of false positives: 364
Number of false negatives: 656
Precision: 0.6388888888888888
Recall: 0.49538461538461537
F measure: 0.5580589254766031
Processing time: 1394.1810930000001
 
Other entries with ngram=(1,2)
New matches: 170
Total matches: 1178
Number of true positives: 758
Number of false positives: 420
Number of false negatives: 542
Precision: 0.6434634974533107
Recall: 0.583076923076923
F measure: 0.6117836965294593
Processing time: 656.8266289999999
 
Other entries with ngram=(1,1)
New matches: 288
Total matches: 1466
Number of true positives: 906
Number of false positives: 560
Number of false negatives: 394
Precision: 0.6180081855388813
Recall: 0.696923076923077
F measure: 0.6550976138828634
Processing time: 64.15175499999987
 
